In [1]:
import numpy as np
import pandas as pd

In [2]:
df_copolymer_real = pd.read_excel('copolymer_real_data_eval_20250820.xlsx')
df_copolymer_gpt4o=pd.read_excel('copolymer_llm_data_eval_20250820.xlsx')

In [3]:
df_copolymer_real = df_copolymer_real[df_copolymer_real['Composition1_ratio'] != 0]
df_copolymer_real = df_copolymer_real[df_copolymer_real['Composition1_ratio'] != 100]
df_copolymer_real = df_copolymer_real[df_copolymer_real['Composition2_ratio'] != 0]
df_copolymer_real = df_copolymer_real[df_copolymer_real['Composition2_ratio'] != 100]

In [4]:
df_copolymer_real =df_copolymer_real.reset_index(drop=True)
df_copolymer_real

,ID,Copolymer name,Component_1_name,Component_2_name,Composition1_ratio,Composition2_ratio,Glass transition temperature,Melting temperature
0,1,ethylene-vinyl acetate copolymer,ethylene,vinyl acetate,70,30,231,345
1,1,ethylene-vinyl acetate copolymer,ethylene,vinyl acetate,60,40,235,318
2,2,ethylene-vinyl alcohol copolymer,ethylene,vinyl alcohol,73,27,345,464
3,2,ethylene-vinyl alcohol copolymer,ethylene,vinyl alcohol,68,32,342,454
4,2,ethylene-vinyl alcohol copolymer,ethylene,vinyl alcohol,62,38,335,448
...,...,...,...,...,...,...,...,...
1972,393,Ethylene/C'11-Br copolymer,Ethylene,11-bromo-1-undecene,97.1,2.9,Nan,116
1973,393,Ethylene/C'11-Br copolymer,Ethylene,11-bromo-1-undecene,99,1,Nan,121
1974,393,Ethylene/C'11-Br copolymer,Ethylene,11-bromo-1-undecene,96.7,3.3,Nan,120
1975,393,Ethylene/C'11-Br copolymer,Ethylene,11-bromo-1-undecene,95.7,4.3,Nan,117


In [5]:
df_copolymer_gpt4o['Copolymer name'] = df_copolymer_gpt4o['Copolymer name'].str.strip()
df_copolymer_gpt4o['Component_1_name'] = df_copolymer_gpt4o['Component_1_name'].str.strip()
df_copolymer_gpt4o['Component_2_name'] = df_copolymer_gpt4o['Component_2_name'].str.strip()

df_copolymer_gpt4o.replace('N/A', pd.NA, inplace=True)
df_copolymer_gpt4o = df_copolymer_gpt4o.dropna(subset=['Copolymer name', 'Component_1_name', 'Component_2_name', 'Composition1_ratio', 'Composition2_ratio', 'Glass transition temperature', 'Melting temperature'], how='all')

df_copolymer_gpt4o = df_copolymer_gpt4o[df_copolymer_gpt4o['Composition1_ratio'] != 0]
df_copolymer_gpt4o = df_copolymer_gpt4o[df_copolymer_gpt4o['Composition1_ratio'] != 100]
df_copolymer_gpt4o = df_copolymer_gpt4o[df_copolymer_gpt4o['Composition2_ratio'] != 0]
df_copolymer_gpt4o = df_copolymer_gpt4o[df_copolymer_gpt4o['Composition2_ratio'] != 100]

df_copolymer_gpt4o = df_copolymer_gpt4o.reset_index(drop=True)

In [6]:
df_copolymer_real= df_copolymer_real.fillna("Nan")
df_copolymer_gpt4o= df_copolymer_gpt4o.fillna("Nan")

In [7]:
df_polymer_name=pd.read_excel(r'D:\AI-Drug\copoly_design\test_pdf\polymer_name_dict.xlsx')
# dict1_to_2 = pd.Series(df_polymer_name.name2.values, index=df_polymer_name.name1).to_dict()
# dict2_to_1 = pd.Series(df_polymer_name.name1.values, index=df_polymer_name.name2).to_dict()
# bidirectional_dict = {**dict1_to_2, **dict2_to_1}
from collections import defaultdict
list1=df_polymer_name['name1'].values.tolist()
list2=df_polymer_name['name2'].values.tolist()
bidirectional_dict = defaultdict(list)
for i in range(len(list1)):
    bidirectional_dict[list1[i]].append(list2[i])
    bidirectional_dict[list2[i]].append(list1[i])

In [8]:
len(bidirectional_dict)

1955

In [9]:
def get_Tg_TP_FP_FN_result(df1,df2):
    TP=0
    FN=0
    FP=0
    for id_value in df1['ID'].unique():
        df1_subset = df1[df1['ID'] == id_value]
        df2_subset = df2[df2['ID'] == id_value]
            
        for _, row1 in df1_subset.iterrows():
            row1_list = row1.to_list()

            found_FN_match = False   
            for _, row2 in df2_subset.iterrows():
                row2_list = row2.to_list()
                Tg_real_final= row1_list[6]
                Tg_gpt_final= row2_list[6]
                if Tg_real_final == Tg_gpt_final:
                    TP=TP+1
                    found_FN_match = True
                    break
            
                
            if not found_FN_match:
                FN = FN+ 1

        for _, row2 in df2_subset.iterrows():
            row2_list = row2.to_list()
            found_FP_match = False
            for _, row1 in df1_subset.iterrows():
                row1_list = row1.to_list()
                Tg_real_final= row1_list[6]
                Tg_gpt_final= row2_list[6]
                if Tg_real_final == Tg_gpt_final:
                    found_FP_match = True
                    break
            if not found_FP_match:
                FP = FP + 1
    recall=TP/(TP+FN)
    precision=TP/(TP+FP)
    F1=2*precision*recall/(precision+recall)
    return TP,FN,FP,recall,precision,F1

In [10]:
get_Tg_TP_FP_FN_result(df_copolymer_real,df_copolymer_gpt4o)

(1390, 587, 546, 0.7030854830551341, 0.7179752066115702, 0.710452338359315)

In [11]:
def get_copolymer_name_TP_FP_FN_result(df1,df2):
    TP=0
    FN=0
    FP=0
    for id_value in df1['ID'].unique():
        df1_subset = df1[df1['ID'] == id_value]
        df2_subset = df2[df2['ID'] == id_value]
            
        for _, row1 in df1_subset.iterrows():
            row1_list = row1.to_list()

            found_FN_match = False   
            for _, row2 in df2_subset.iterrows():
                row2_list = row2.to_list()
                copolymer_name_real_final= row1_list[1]
                copolymer_name_gpt_final= row2_list[1]
                if copolymer_name_real_final == copolymer_name_gpt_final:
                    TP=TP+1
                    found_FN_match = True
                    break

                if copolymer_name_gpt_final in bidirectional_dict:
                    for name in bidirectional_dict[copolymer_name_gpt_final]:
                        if  copolymer_name_real_final==name:
                            TP=TP+1
                            found_FN_match = True 
                            break  
                
            if not found_FN_match:
                FN = FN+ 1
        for _, row2 in df2_subset.iterrows():
            row2_list = row2.to_list()
            found_FP_match = False
            for _, row1 in df1_subset.iterrows():
                row1_list = row1.to_list()
                copolymer_name_real_final= row1_list[1]
                copolymer_name_gpt_final= row2_list[1]
                if copolymer_name_real_final == copolymer_name_gpt_final:
                    found_FP_match = True
                    break

                if copolymer_name_gpt_final in bidirectional_dict:
                    for name in bidirectional_dict[copolymer_name_gpt_final]:
                        if  copolymer_name_real_final==name:
                            found_FP_match = True 
                            break
                    
            if not found_FP_match:
                FP = FP + 1
    recall=TP/(TP+FN)
    precision=TP/(TP+FP)
    F1=2*precision*recall/(precision+recall)
    return TP,FN,FP,recall,precision,F1

def get_Component_1_name_TP_FP_FN_result(df1,df2):
    TP=0
    FN=0
    FP=0
    for id_value in df1['ID'].unique():
        df1_subset = df1[df1['ID'] == id_value]
        df2_subset = df2[df2['ID'] == id_value]
            
        for _, row1 in df1_subset.iterrows():
            row1_list = row1.to_list()

            found_FN_match = False   
            for _, row2 in df2_subset.iterrows():
                row2_list = row2.to_list()
                Component_1_name_real_final= row1_list[2]
                Component_1_name_gpt_final= row2_list[2]
                if Component_1_name_real_final == Component_1_name_gpt_final:
                    TP=TP+1
                    found_FN_match = True
                    break

                if Component_1_name_gpt_final in bidirectional_dict:
                    for name in bidirectional_dict[Component_1_name_gpt_final]:
                        if  Component_1_name_real_final==name:
                            TP=TP+1
                            found_FN_match = True 
                            break  
                
            if not found_FN_match:
                FN = FN+ 1

        for _, row2 in df2_subset.iterrows():
            row2_list = row2.to_list()
            found_FP_match = False
            for _, row1 in df1_subset.iterrows():
                row1_list = row1.to_list()
                Component_1_real_final= row1_list[2]
                Component_1_gpt_final= row2_list[2]
                if Component_1_real_final == Component_1_gpt_final:
                    found_FP_match = True
                    break

                if Component_1_gpt_final in bidirectional_dict:
                    for name in bidirectional_dict[Component_1_gpt_final]:
                        if  Component_1_real_final==name:
                            found_FP_match = True 
                            break

            if not found_FP_match:
                FP = FP + 1
    recall=TP/(TP+FN)
    precision=TP/(TP+FP)
    F1=2*precision*recall/(precision+recall)
    return TP,FN,FP,recall,precision,F1   
 

def get_Component_2_name_TP_FP_FN_result(df1,df2):
    TP=0
    FN=0
    FP=0
    for id_value in df1['ID'].unique():
        df1_subset = df1[df1['ID'] == id_value]
        df2_subset = df2[df2['ID'] == id_value]
            
        for _, row1 in df1_subset.iterrows():
            row1_list = row1.to_list()

            found_FN_match = False   
            for _, row2 in df2_subset.iterrows():
                row2_list = row2.to_list()
                Component_2_name_real_final= row1_list[3]
                Component_2_name_gpt_final= row2_list[3]
                if Component_2_name_real_final == Component_2_name_gpt_final:
                    TP=TP+1
                    found_FN_match = True
                    break

                if Component_2_name_gpt_final in bidirectional_dict:
                    for name in bidirectional_dict[Component_2_name_gpt_final]:
                        if  Component_2_name_real_final==name:
                            TP=TP+1
                            found_FN_match = True 
                            break  
                
            if not found_FN_match:
                FN = FN+ 1
        for _, row2 in df2_subset.iterrows():
            row2_list = row2.to_list()
            found_FP_match = False
            for _, row1 in df1_subset.iterrows():
                row1_list = row1.to_list()
                Component_2_real_final= row1_list[3]
                Component_2_gpt_final= row2_list[3]
                if Component_2_real_final == Component_2_gpt_final:
                    found_FP_match = True
                    break

                if Component_2_gpt_final in bidirectional_dict:
                    for name in bidirectional_dict[Component_2_gpt_final]:
                        if  Component_2_real_final==name:
                            found_FP_match = True 
                            break

            if not found_FP_match:
                FP = FP + 1
    recall=TP/(TP+FN)
    precision=TP/(TP+FP)
    F1=2*precision*recall/(precision+recall)
    return TP,FN,FP,recall,precision,F1  

def get_Composition1_ratio_TP_FP_FN_result(df1,df2):
    TP=0
    FN=0
    FP=0
    for id_value in df1['ID'].unique():
        df1_subset = df1[df1['ID'] == id_value]
        df2_subset = df2[df2['ID'] == id_value]
            
        for _, row1 in df1_subset.iterrows():
            row1_list = row1.to_list()

            found_match = False   
            for _, row2 in df2_subset.iterrows():
                row2_list = row2.to_list()
                Composition1_ratio_real_final= row1_list[4]
                Composition1_ratio_gpt_final= row2_list[4]
                if Composition1_ratio_real_final == Composition1_ratio_gpt_final:
                    TP=TP+1
                    found_match = True
                    break
                
            if not found_match:
                FN = FN+ 1
        for _, row2 in df2_subset.iterrows():
            row2_list = row2.to_list()
            found_FP_match = False
            for _, row1 in df1_subset.iterrows():
                row1_list = row1.to_list()
                Composition1_real_final= row1_list[4]
                Composition1_gpt_final= row2_list[4]
                if Composition1_real_final == Composition1_gpt_final:
                    found_FP_match = True
                    break
            if not found_FP_match:
                FP = FP + 1
    recall=TP/(TP+FN)
    precision=TP/(TP+FP)
    F1=2*precision*recall/(precision+recall)
    return TP,FN,FP,recall,precision,F1    

def get_Composition2_ratio_TP_FP_FN_result(df1,df2):
    TP=0
    FN=0
    FP=0
    for id_value in df1['ID'].unique():
        df1_subset = df1[df1['ID'] == id_value]
        df2_subset = df2[df2['ID'] == id_value]
            
        for _, row1 in df1_subset.iterrows():
            row1_list = row1.to_list()

            found_match = False   
            for _, row2 in df2_subset.iterrows():
                row2_list = row2.to_list()
                Composition2_ratio_real_final= row1_list[5]
                Composition2_ratio_gpt_final= row2_list[5]
                if Composition2_ratio_real_final == Composition2_ratio_gpt_final:
                    TP=TP+1
                    found_match = True
                    break
                
            if not found_match:
                FN = FN+ 1
        for _, row2 in df2_subset.iterrows():
            row2_list = row2.to_list()
            found_FP_match = False
            for _, row1 in df1_subset.iterrows():
                row1_list = row1.to_list()
                Composition2_real_final= row1_list[5]
                Composition2_gpt_final= row2_list[5]
                if Composition2_real_final == Composition2_gpt_final:
                    found_FP_match = True
                    break
            if not found_FP_match:
                FP = FP + 1
    recall=TP/(TP+FN)
    precision=TP/(TP+FP)
    F1=2*precision*recall/(precision+recall)
    return TP,FN,FP,recall,precision,F1   

def get_Tm_TP_FP_FN_result(df1,df2):
    TP=0
    FN=0
    FP=0
    for id_value in df1['ID'].unique():
        df1_subset = df1[df1['ID'] == id_value]
        df2_subset = df2[df2['ID'] == id_value]
            
        for _, row1 in df1_subset.iterrows():
            row1_list = row1.to_list()

            found_FN_match = False   
            for _, row2 in df2_subset.iterrows():
                row2_list = row2.to_list()
                Tm_real_final= row1_list[7]
                Tm_gpt_final= row2_list[7]
                if Tm_real_final == Tm_gpt_final:
                    TP=TP+1
                    found_FN_match = True
                    break
                
            if not found_FN_match:
                FN = FN+ 1

        for _, row2 in df2_subset.iterrows():
            row2_list = row2.to_list()
            found_FP_match = False
            for _, row1 in df1_subset.iterrows():
                row1_list = row1.to_list()
                Tm_real_final= row1_list[7]
                Tm_gpt_final= row2_list[7]
                if Tm_real_final == Tm_gpt_final:
                    found_FP_match = True
                    break
            if not found_FP_match:
                FP = FP + 1
    recall=TP/(TP+FN)
    precision=TP/(TP+FP)
    F1=2*precision*recall/(precision+recall)
    return TP,FN,FP,recall,precision,F1              

In [12]:
copolymer_name_TP,copolymer_name_FN,copolymer_name_FP,copolymer_name_reacll,copolymer_name_precision,copolymer_name_F1=get_copolymer_name_TP_FP_FN_result(df_copolymer_real,df_copolymer_gpt4o)
Component_1_name_TP,Component_1_name_FN,Component_1_name_FP,Component_1_name_reacll,Component_1_name_precision,Component_1_name_F1=get_Component_1_name_TP_FP_FN_result(df_copolymer_real,df_copolymer_gpt4o)
Component_2_name_TP,Component_2_name_FN,Component_2_name_FP,Component_2_name_reacll,Component_2_name_precision,Component_2_name_F1=get_Component_2_name_TP_FP_FN_result(df_copolymer_real,df_copolymer_gpt4o)
Composition1_TP,Composition1_FN,Composition1_FP,Composition1_reacll,Composition1_precision,Composition1_F1=get_Composition1_ratio_TP_FP_FN_result(df_copolymer_real,df_copolymer_gpt4o)
Composition2_TP,Composition2_FN,Composition2_FP,Composition2_reacll,Composition2_precision,Composition2_F1=get_Composition2_ratio_TP_FP_FN_result(df_copolymer_real,df_copolymer_gpt4o)
Tg_TP,Tg_FN,Tg_FP,Tg_reacll,Tg_precision,Tg_F1=get_Tg_TP_FP_FN_result(df_copolymer_real,df_copolymer_gpt4o)
Tm_TP,Tm_FN,Tm_FP,Tm_reacll,Tm_precision,Tm_F1=get_Tm_TP_FP_FN_result(df_copolymer_real,df_copolymer_gpt4o)

In [13]:
eval_data = {
    'Metric': ['Copolymer Name', 'Component 1 Name', 'Component 2 Name', 'Composition 1 ratio', 'Composition 2 ratio', 'Tg', 'Tm'],
    'Recall': [copolymer_name_reacll, Component_1_name_reacll, Component_2_name_reacll, Composition1_reacll, Composition2_reacll, Tg_reacll, Tm_reacll],
    'Precision': [copolymer_name_precision, Component_1_name_precision, Component_2_name_precision, Composition1_precision, Composition2_precision, Tg_precision, Tm_precision],
    'F1-Score': [copolymer_name_F1, Component_1_name_F1, Component_2_name_F1, Composition1_F1, Composition2_F1, Tg_F1, Tm_F1]
}

df_eval = pd.DataFrame(eval_data)

In [14]:
df_eval

,Metric,Recall,Precision,F1-Score
0,Copolymer Name,0.899132,0.899132,0.899132
1,Component 1 Name,0.872048,0.876293,0.874165
2,Component 2 Name,0.865554,0.862265,0.863907
3,Composition 1 ratio,0.702580,0.732595,0.717273
4,Composition 2 ratio,0.709155,0.732497,0.720637
5,Tg,0.703085,0.717975,0.710452
6,Tm,0.689934,0.706736,0.698234


In [15]:
df_eval.describe()

,Recall,Precision,F1-Score
count,7.000000,7.000000,7.000000
mean,0.777356,0.789642,0.783400
std,0.095722,0.084951,0.090370
min,0.689934,0.706736,0.698234
25%,0.702833,0.725236,0.713863
50%,0.709155,0.732595,0.720637
75%,0.868801,0.869279,0.869036
max,0.899132,0.899132,0.899132
